# Задание по программированию: 1NN против RandomForest

В этом задании будет использоваться датасет **digits** из **sklearn.datasets**. Оставьте последние **25%** объектов для контроля качества, разделив X и y на **X_train, y_train и X_test, y_test**.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами **1NN** с **RandomForestClassifier** из sklearn на **1000 деревьях**.

## Задание 1

Реализуйте самостоятельно **метод одного ближайшего соседа с евклидовой метрикой для задачи классификации**. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.  

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: **можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием)**.  

In [13]:
from sklearn import datasets, model_selection, metrics
import numpy as np
import math

data = datasets.load_digits()
X_train, X_test, y_train, y_test = model_selection.train_test_split(data.data, data.target, test_size = 0.25, random_state = 0)

In [63]:
#a = [0,0,0]
#b = [2,2,2]
#print(sum([(a[0]- a[1])**2 for a in zip(a,b)]))

In [48]:
%%time
y_predicted = np.zeros(len(X_test))

for i in range(len(X_test)):
    min_index = -1 
    min_dist = float('inf') 
    for j in range(len(X_train)):
        dist = sum([(a[0]- a[1])**2 for a in zip(X_test[i],X_train[j])])
        if dist < min_dist:
            min_dist = dist
            min_index = j
            
    y_predicted[i] = y_train[min_index]

CPU times: user 35.7 s, sys: 0 ns, total: 35.7 s
Wall time: 35.7 s


In [62]:
from sklearn.metrics import accuracy_score

score = accuracy_score(y_predicted, y_test)
print("accuracy_score", score)


errors_cnt = len(list(filter(lambda a: a[0] != a[1] , zip(y_predicted, y_test))))
print("error rate", errors_cnt / len(y_test))


accuracy_score 0.9911111111111112
error rate 0.008888888888888889


Сортировка массива длиной N требует порядка **N log N** сравнений (строже говоря, она работает за **O(N log N))**. **Подумайте, как можно легко улучшить получившееся время работы**.   

Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться **со структурами данных ball tree и kd tree.**

**Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.**

## Задание 2

Теперь обучите на обучающей выборке **RandomForestClassifier(n_estimators=1000)** из sklearn.   

Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней.   

**Эта доля — ответ в задании 2.**


Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. 

Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [49]:
from sklearn import ensemble, model_selection, metrics 

In [65]:
%%time
rf_classifier = ensemble.RandomForestClassifier(n_estimators = 1000, random_state = 1)
rf_classifier.fit(X_train, y_train)
y_predicted_rf = rf_classifier.predict(X_test)

CPU times: user 2.84 s, sys: 7.99 ms, total: 2.85 s
Wall time: 2.85 s


In [68]:
score = accuracy_score(y_predicted_rf, y_test)
print("accuracy_score", score)

errors_cnt = len(list(filter(lambda a: a[0] != a[1] , zip(y_predicted_rf, y_test))))
print("error rate", errors_cnt / len(y_test))

accuracy_score 0.98
error rate 0.02
